Этот код позволяет найти контактные данные (телефоны, почты, ссылки и проч.) в .pdf / .docx файлах, расположенных в директории Google Drive.

Чтобы использовать:
- залей файлы в [облачную папку](https://drive.google.com/drive/folders/1BnT3wK8owapiP-mxxhn-9Kt4e7uvBZRX)
- нажми кнопку запуска
- разреши авторизацию Google OAuth во всплывающем окне
- подожди несколько минут
- забери результат из загрузок браузера

In [ ]:
# TODO: подвести сервисный аккаунт вместо IAM-токена / реализовать переполучение IAM-токена cURL'ами

# Подгружаем инструкции

In [ ]:
%%capture
!pip install -U -q PyDrive@1.19.0

In [ ]:
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Скачаем файлы из папки Google Drive в локальное временное хранилище
%%capture
local_download_path = os.path.expanduser('/content')
try:
  os.makedirs(local_download_path)
except: pass

file_list = drive.ListFile(
    {'q': "'<ID папки Google Drive из URL' in parents"}).GetList()

for f in file_list:

  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


try:
  with open(fname, 'r') as f:
    print(f.read())
except UnicodeDecodeError:
  pass

In [ ]:
directory_path = '/content/'
directory_files = os.listdir(directory_path)

files = [x for x in directory_files if ".pdf" in x or ".docx" in x]
print(files)

['PUBID_345395-Инструкция по обработке обслуживания (Релиз 5.1).pdf', 'PUBID_2009524-Руководство пользователя AI.pdf', 'PUBID_345395-Список изменений.pdf']


# Подключаем инструменты, авторизуемся в Yandex Cloud

In [ ]:
%%capture
!pip install pdfminer==20191125

In [ ]:
%%capture
!pip install Pillow==10.3.0

In [ ]:
%%capture
!pip install PyMuPDF==1.24.0

In [ ]:
%%capture
# Установим русскоязычный датасет Tesseract
!sudo apt-get install tesseract-ocr-rus

In [ ]:
%%capture
!pip install pytesseract==0.3.10

In [ ]:
%%capture
!pip install spacy==3.7.4

In [ ]:
%%capture
!pip install python-docx==1.1.0

In [ ]:
import pytesseract as pt

# Укажем путь до исполняемого файла Tesseract
pt.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [ ]:
from docx import Document
from PIL import Image
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from spacy.lang.ru import Russian
from google.colab import userdata
from google.colab import files as fls

import fitz
import glob
import io
import json
import shutil
import requests
import time

In [ ]:
# Подгрузим креды Yandex Cloud
FOLDER_ID = userdata.get('FOLDER_ID')
IAM_TOKEN = userdata.get('IAM_TOKEN')
# API_TOKEN = userdata.get('API_TOKEN')

# Зададим заголовок POST-запроса к YandexGPT
# headers = {'Authorization': f'Api-Key {API_TOKEN}'}
headers = {'Authorization': f'Bearer {IAM_TOKEN}'}

# Создаем рабочие папки

In [ ]:
# Создадим временную папку для хранения извлеченных картинок
!mkdir to_extract

# Создадим временную папку для хранения извлеченного из текста на русском картинок
!mkdir result_rus

# Создадим временную папку для хранения извлеченного из текста на английском картинок
!mkdir result_eng

# Создадим папку для хранения результата проверки YaGPT
!mkdir prompts

# Задаем функции извлечения текста и картинок

In [ ]:
# Извлечем кириллический текст из документа
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()

    if text:
        return text

In [ ]:
def extract_text_from_docx(docx_path):
  f = open(f'{docx_path}', 'rb')
  document = Document(f)
  fullText = []
  for para in document.paragraphs:
      fullText.append(para.text)
  return '\n'.join(fullText)

In [ ]:
# Извлечем текст из картинок на русском языке
def extract_rus_text_from_images():
  imagePath ="/content/to_extract"
  tempPath ="/content/result_rus"

  for imageName in os.listdir(imagePath):
    try:
      inputPath = os.path.join(imagePath, imageName)
      img = Image.open(inputPath)

      text = pt.image_to_string(img, lang ="rus")

      fullTempPath = os.path.join(tempPath, imageName + ".txt")

      # Сохраним результат в отдельные файлы
      file1 = open(fullTempPath, "w")
      print(f"Текст на изображении распознан.")
      file1.write(text)
      file1.close()
    except IsADirectoryError:
      pass

In [ ]:
# Извлечем текст из картинок на английском языке
def extract_eng_text_from_images():
  imagePath ="/content/to_extract"
  tempPath ="/content/result_eng"

  for imageName in os.listdir(imagePath):
    try:
      inputPath = os.path.join(imagePath, imageName)
      img = Image.open(inputPath)

      text = pt.image_to_string(img, lang ="rus")

      fullTempPath = os.path.join(tempPath, imageName + ".txt")

      # Сохраним результат в отдельные файлы
      file1 = open(fullTempPath, "w")
      print(f"Текст на изображении распознан.")
      file1.write(text)
      file1.close()
    except IsADirectoryError:
      pass

In [ ]:
# Извлечем текст из картинок
def extract_text_from_images():
  images_to_extract = os.listdir('/content/to_extract')

  if len(images_to_extract) > 0:
    extract_rus_text_from_images()
    extract_eng_text_from_images()

    tempEngPath = "/content/result_eng/"
    engTemp = os.listdir(tempEngPath)
    engFiles = [tempEngPath + s for s in engTemp]

    tempRusPath = "/content/result_rus/"
    rusTemp = os.listdir(tempRusPath)
    rusFiles = [tempRusPath + s for s in rusTemp]

    # Сохраним текст из картинок на английском в отдельный файл
    with open('output_eng.txt','wb') as wfd:
        for f in engFiles:
            try:
              with open(f,'rb') as fd:
                  shutil.copyfileobj(fd, wfd)
            except IsADirectoryError:
              pass

    # Сохраним текст из картинок на русском в отдельный файл
    with open('output_rus.txt','wb') as wfd:
        for f in rusFiles:
            try:
              with open(f,'rb') as fd:
                  shutil.copyfileobj(fd, wfd)
            except IsADirectoryError:
              pass
  else:
    pass

In [ ]:
# Выделим контактные данные из текста
def search_for_contacts(auth_headers, FILENAME):
  # Объединим файлы: текст инструкции (без картинок), распознанный с картинок текст на английском + русском
  filenames = [f'{FILENAME}_output.txt', f'output_eng.txt', f'output_rus.txt']

  with open(f'/content/{FILENAME}_output_total.txt', 'w') as outfile:
      for fname in filenames:
          with open(fname) as infile:
              outfile.write(infile.read())

  # Разложим текст на токены
  with open(f'/content/{FILENAME}_output_total.txt') as f:
    contents = f.read()

  # Разделим итоговый текст на пакеты токенов
  nlp = Russian()
  doc = nlp(contents)

  batches_with_duplicates = [token.text for token in doc]
  batches = list(set(batches_with_duplicates))


  # Определим число запросов к YandexGPT API
  batchesModulo = len(batches) % 1000

  if batchesModulo > 0:
    batchesQty = int(len(batches) / 1000) + 1
  else:
    batchesQty = int(len(batches) / 1000)

  print(f'Обращений к YaGPT: {batchesQty}')

  i = 1

  while i <= batchesQty:
    batchMax = 1000 * i
    batchMin = batchMax - 999
    batch = batches[batchMin:batchMax]

    i += 1

    prompt = {
      "modelUri": f"gpt://{FOLDER_ID}/yandexgpt-lite",
      "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "1100"
      },
      "messages": [
        {
          "role": "system",
          "text": "Перечисли через запятую контактные данные (телефоны, почты, ссылки, токены), которые есть в приложенном тексте . Названия компаний и брендов не перечисляй как возможные контактные данные. Больше ничего не пиши."
        },
        {
          "role": "user",
          "text": f"{batch}"
        }
      ]
    }

    with open(f'prompts/prompt_{i-1}.json', 'w') as f:
        json.dump(prompt, f, ensure_ascii=False)

  url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'

  with open(f'prompts/prompt_{i-1}.json', 'r', encoding='utf-8') as f:
      data = json.dumps(json.load(f))
  resp = requests.post(url, headers=auth_headers, data=data)
  print(json.loads(resp.text)["result"]["alternatives"][0]["message"]["text"])
  return json.loads(resp.text)["result"]["alternatives"][0]["message"]["text"]

In [ ]:
def get_pixmaps_in_pdf(pdf_filename):
    doc = fitz.open(pdf_filename)
    xrefs = set()
    for page_index in range(doc.page_count):
        for image in doc.get_page_images(page_index):
            xrefs.add(image[0])
    pixmaps = [fitz.Pixmap(doc, xref) for xref in xrefs]
    doc.close()
    return pixmaps


def write_pixmaps_to_pngs(pixmaps):
    for i, pixmap in enumerate(pixmaps):
        pixmap.save(open(f"to_extract/image_{time.time()}.png", "wb"))

# Извлекаем контакты

In [ ]:
# Извлекаем текст
i = 0

while i < len(files):
  FILE = f'/content/{files[i]}'
  print(files[0])
  try:
    text = extract_text_from_pdf(FILE)
    print(f"Текст извлечен из файла: {files[i]}")
  except Exception as e:
    text = extract_text_from_docx(FILE)
    print(f"Текст извлечен из файла: {files[i]}")

  with open(f"{files[i]}_output.txt", "w+") as f:
      f.writelines(text)

  i += 1

PUBID_345395-Инструкция по обработке обслуживания (Релиз 5.1).pdf
Текст извлечен из файла: PUBID_345395-Инструкция по обработке обслуживания (Релиз 5.1).pdf
PUBID_345395-Инструкция по обработке обслуживания (Релиз 5.1).pdf
Текст извлечен из файла: PUBID_2009524-Руководство пользователя AI.pdf
PUBID_345395-Инструкция по обработке обслуживания (Релиз 5.1).pdf
Текст извлечен из файла: PUBID_345395-Список изменений.pdf


In [ ]:
# Извлекаем картинки
i = 0

while i < len(files):
  pixmaps = get_pixmaps_in_pdf(files[i])
  write_pixmaps_to_pngs(pixmaps)
  print(f"Изображения извлечены из файлов.")
  i += 1

Изображения извлечены из файлов.
Изображения извлечены из файлов.
Изображения извлечены из файлов.


In [ ]:
extract_text_from_images()
print(f"Текст извлечен из картинок.")

In [ ]:
i = 0

while i < len(files):
  print(search_for_contacts(headers, files[i]))

  # Зальем результат в отдельный файл
  f = open(f"{files[i]}_result.txt", "a")
  print(f"Контакты сохранены в файл: {files[i]}_result.txt")

  # Скачаем результат поиска
  f.close()
  fls.download(f"{files[i]}_result.txt")
  i += 1

In [ ]:
!%%capture
# Удалим временные папки и файлы
!rm -rf prompts
!rm -rf result_eng
!rm -rf result_rus
!rm -rf to_extract